<a href="https://colab.research.google.com/github/subramanya4shenoy/MachineLearningNbs/blob/main/playground_series_s5e1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# prompt: connect to a kaggle playground seriese competition

# Install the Kaggle API client.
!pip install kaggle

# Upload your kaggle.json file (contains API key and secret).
from google.colab import files
files.upload()

# Make a directory for the Kaggle API key.
!mkdir -p ~/.kaggle

# Move the kaggle.json file to the Kaggle API directory.
!mv kaggle.json ~/.kaggle/

# Set permissions for the Kaggle API key file.
!chmod 600 ~/.kaggle/kaggle.json

# Download the competition data. Replace 'YOUR_COMPETITION_NAME' with the actual name.
!kaggle competitions download -c playground-series-s5e1

# Unzip the data.  Adjust the file name if it's different.
!unzip playground-series-s5e1.zip

# Now you can access the competition data in the current directory.

Saving kaggle.json to kaggle.json
playground-series-s5e1.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  playground-series-s5e1.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission.csv   
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


#Imports and loding the data

In [11]:
import pandas as pd
import numpy as np

In [12]:
data = pd.read_csv("/content/train.csv", index_col='id')

# EDA Steps


*   6 columns: 1 - index column, 1 target column (num_sold), 4 training columns
*   predict 1 colunm from given



In [16]:
data.shape

(230130, 5)

In [13]:
data.head()

,date,country,store,product,num_sold
id,,,,,
0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


In [14]:
data.dtypes

,0
date,object
country,object
store,object
product,object
num_sold,float64


## Columns


* all 4 string tye columns
* Dont see a use case for scalar



### Handling 'Country' columns

In [24]:
data_copy = data.copy()

In [25]:
data_copy['country'].value_counts()

,count
country,
Canada,38355
Finland,38355
Italy,38355
Kenya,38355
Norway,38355
Singapore,38355


In [26]:
data_copy['country'].isna().sum()

0

### ⬆ Simple Label classifier for country

### Handling 'Store' columns

In [31]:
data_copy = data.copy()

In [32]:
data_copy['store'].value_counts()

,count
store,
Discount Stickers,76710
Stickers for Less,76710
Premium Sticker Mart,76710


In [33]:
data_copy['store'].value_counts()

,count
store,
Discount Stickers,76710
Stickers for Less,76710
Premium Sticker Mart,76710


In [34]:
data_copy['store'].isna().sum()

0

### ⬆ Simple Label classifier for Store

### Handling 'Product' columns

In [35]:
data_copy = data.copy()

In [36]:
data_copy['product'].value_counts()

,count
product,
Holographic Goose,46026
Kaggle,46026
Kaggle Tiers,46026
Kerneler,46026
Kerneler Dark Mode,46026


In [37]:
data_copy['product'].isna().sum()

0

### ⬆ Simple Label classifier for Product

### Handling 'date' columns

In [39]:
data_copy = data.copy()

In [40]:
data['date'].value_counts()

,count
date,
2010-01-01,90
2014-09-05,90
2014-08-29,90
2014-08-30,90
2014-08-31,90
...,...
2012-05-03,90
2012-05-04,90
2012-05-05,90


In [41]:
data['date'].isna().sum()

0

In [42]:
# prompt: how do we rtepresent a date column while converting it into a numbers ?

# Convert the 'date' column to datetime objects
data_copy['date'] = pd.to_datetime(data_copy['date'])

# Extract relevant date features (year, month, day)
data_copy['year'] = data_copy['date'].dt.year
data_copy['month'] = data_copy['date'].dt.month
data_copy['day'] = data_copy['date'].dt.day

# You can also create other date-related features like day of week, week of year, etc.
data_copy['dayofweek'] = data_copy['date'].dt.dayofweek
data_copy['weekofyear'] = data_copy['date'].dt.isocalendar().week

# Optionally, you can drop the original 'date' column if no longer needed
data_copy = data_copy.drop('date', axis=1)

# Now you have numerical representations of the date in separate columns
print(data_copy.head())

   country              store             product  num_sold  year  month  day  \
id                                                                              
0   Canada  Discount Stickers   Holographic Goose       NaN  2010      1    1   
1   Canada  Discount Stickers              Kaggle     973.0  2010      1    1   
2   Canada  Discount Stickers        Kaggle Tiers     906.0  2010      1    1   
3   Canada  Discount Stickers            Kerneler     423.0  2010      1    1   
4   Canada  Discount Stickers  Kerneler Dark Mode     491.0  2010      1    1   

    dayofweek  weekofyear  
id                         
0           4          53  
1           4          53  
2           4          53  
3           4          53  
4           4          53  


# Preprocessing

In [43]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [44]:
labelEncoder = LabelEncoder()

In [45]:
def tranform_categorical_cols(data):
  label_encoder_columns = ['country', 'store', 'product']
  for column in label_encoder_columns:
    data[column] = labelEncoder.fit_transform(data[column])

  return data

In [53]:
def transform_date_cols(data):
  # Convert the 'date' column to datetime objects
  data['date'] = pd.to_datetime(data['date'])

  # Extract relevant date features (year, month, day)
  data['year'] = data['date'].dt.year
  data['month'] = data['date'].dt.month
  data['day'] = data['date'].dt.day

  # You can also create other date-related features like day of week, week of year, etc.
  data['dayofweek'] = data['date'].dt.dayofweek
  data['weekofyear'] = data['date'].dt.isocalendar().week

  # Optionally, you can drop the original 'date' column if no longer needed
  data = data.drop('date', axis=1)

  return data

In [58]:
def transform_numerical_cols(data):
  data = transform_date_cols(data)
  return data

In [133]:
def preprocess_data(data):
  data = tranform_categorical_cols(data)
  data = transform_numerical_cols(data)
  return data

In [141]:
data_copy = preprocess_data(data)
data_copy['num_sold'].fillna(0, inplace=True)

<ipython-input-141-28c6689b8059>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_copy['num_sold'].fillna(0, inplace=True)


# Training the model

In [164]:
data_copy = preprocess_data(data)
data_copy['num_sold'].fillna(0, inplace=True)

<ipython-input-164-28c6689b8059>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_copy['num_sold'].fillna(0, inplace=True)


In [165]:
X = data_copy.drop('num_sold', axis=1)
y = data_copy['num_sold']

In [166]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [167]:
scaler = StandardScaler()

In [168]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [169]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [170]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [171]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                     │ (None, 128)                 │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,473 (37.00 KB)

 Trainable params: 9,473 (37.00 KB)

 Non-trainable params: 0 (0.00 B)

In [172]:
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

In [173]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [174]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/100
5754/5754 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 477.0948 - mae: 477.0948 - val_loss: 281.1295 - val_mae: 281.1295
Epoch 2/100
5754/5754 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 252.8582 - mae: 252.8582 - val_loss: 178.7760 - val_mae: 178.7760
Epoch 3/100
5754/5754 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 158.9384 - mae: 158.9384 - val_loss: 114.1920 - val_mae: 114.1920
Epoch 4/100
5754/5754 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 106.8534 - mae: 106.8534 - val_loss: 94.4233 - val_mae: 94.4233
Epoch 5/100
5754/5754 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 93.0920 - mae: 93.0920 - val_loss: 87.9284 - val_mae: 87.9284
Epoch 6/100
5754/5754 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 86.1665 - mae: 86.1665 - val_loss: 84.5099 - val_mae: 84.5099
Epoch 7/100
5754/5754 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 81.9287 - mae: 81.9287 - val_loss: 80.4097 - val_mae: 80.4097
Epoch 8/100
5754/5754 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 78.5724 - mae: 78.5724 - val_loss: 76.56

In [175]:
model.save('model_ann_2.h5')

# Prediction

In [184]:
test = pd.read_csv("/content/test.csv", index_col='id')

In [185]:
ids = test.index

In [186]:
test = preprocess_data(test)

In [187]:
test = scaler.transform(test)

In [188]:
prediction = model.predict(test)

3080/3080 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step


In [189]:
submit = pd.DataFrame({'id': ids, 'num_sold': prediction.flatten()})

In [190]:
submit.to_csv('submission2.csv', index=False)